# **Chapter 1: Working with Text Data**


### **2.1 Tokenizing Text**

In [109]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [110]:
with open("the-verdict.txt","r", encoding="utf-8") as f :
    raw_text = f.read()

In [111]:
len(raw_text)

20479

### warm up text just for example

In [112]:
import re # python regular operation

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

result

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']

In [113]:
result = re.split(r'([,.]|\s)', text)

In [114]:
result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'This',
 ',',
 '',
 ' ',
 'is',
 ' ',
 'a',
 ' ',
 'test',
 '.',
 '']

In [115]:
# to get rid of the white spaced charcters(spaces in between)
result = [item for item in result if item.strip()]
result

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

In [116]:
# more complex example
text = "Hello, world. Is this -- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s])', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is this', '--', 'a test', '?']


### Now we use our actual data

In [117]:
result = re.split(r'([,.:;?_!()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
preprocessed = result

In [118]:
len(preprocessed)

4606

In [119]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

# **2.2 Converting the tokens into token IDs**

In [120]:
all_words = sorted(set(preprocessed))   # 'set' will help us to get rid of the duplicates in the data 
all_words[:20]

['!',
 '"',
 '"Ah',
 '"Be',
 '"Begin',
 '"By',
 '"Come',
 '"Destroyed',
 '"Don',
 '"Gisburns"',
 '"Grindles',
 '"Hang',
 '"Has',
 '"How',
 '"I',
 '"If',
 '"It',
 '"Jack',
 '"Money',
 '"Moon-dancers"']

In [121]:
vocab_size = len(all_words)
vocab_size

1158

In [122]:
# lets build the vacabulary for each token
vocab = {token : integer for integer, token in enumerate(all_words)}
vocab
# it will enumarate all the words and assign unique integer in asscending order
# now we use this vocabulary to tokenize training data to token ids

{'!': 0,
 '"': 1,
 '"Ah': 2,
 '"Be': 3,
 '"Begin': 4,
 '"By': 5,
 '"Come': 6,
 '"Destroyed': 7,
 '"Don': 8,
 '"Gisburns"': 9,
 '"Grindles': 10,
 '"Hang': 11,
 '"Has': 12,
 '"How': 13,
 '"I': 14,
 '"If': 15,
 '"It': 16,
 '"Jack': 17,
 '"Money': 18,
 '"Moon-dancers"': 19,
 '"Mr': 20,
 '"Mrs': 21,
 '"My': 22,
 '"Never': 23,
 '"Of': 24,
 '"Oh': 25,
 '"Once': 26,
 '"Only': 27,
 '"Or': 28,
 '"That': 29,
 '"The': 30,
 '"Then': 31,
 '"There': 32,
 '"This': 33,
 '"We': 34,
 '"Well': 35,
 '"What': 36,
 '"When': 37,
 '"Why': 38,
 '"Yes': 39,
 '"You': 40,
 '"but': 41,
 '"deadening': 42,
 '"dragged': 43,
 '"effects"': 44,
 '"interesting"': 45,
 '"lift': 46,
 '"obituary"': 47,
 '"strongest': 48,
 '"strongly"': 49,
 '"sweetly"': 50,
 "'": 51,
 '(': 52,
 ')': 53,
 ',': 54,
 '--': 55,
 '.': 56,
 ':': 57,
 ';': 58,
 '?': 59,
 'A': 60,
 'Among': 61,
 'And': 62,
 'Are': 63,
 'Arrt': 64,
 'As': 65,
 'At': 66,
 'Burlington': 67,
 'But': 68,
 'By': 69,
 'Carlo': 70,
 'Chicago': 71,
 'Claude': 72,
 'Croft': 7

In [123]:
# Tokenizing the same thing in raw python
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i : s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'[,.:;?_!"()\']|--|\s', text)

        preprocessed = [item.strip() for item in preprocessed if item and item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+(,.?!"()\'])', r'\1', text)
        return text

In [124]:
tokenizer = SimpleTokenizerV1(vocab)

In [125]:
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""

In [126]:
ids = tokenizer.encode(text)
ids 

[95, 880, 1015, 633, 564, 776, 1154, 627, 104, 82, 881, 1136, 784, 823]

In [127]:
tokenizer.decode(ids)

'It s the last he painted you know Mrs Gisburn said with pardonable pride'

### **2.4 Adding Special context Tokens** 

In [128]:
text = "Hello, Do you like tea. is this-- a test?"

tokenizer.encode(text)

# here 'Hello' is not in the actual data so this is raising an error so to tackle this issue we will add soecial context tokens

KeyError: 'Hello'

In [129]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}


In [130]:
len(vocab.items())

1160

In [131]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1155)
('your', 1156)
('yourself', 1157)
('<|endoftext|>', 1158)
('<|unk|>', 1159)


In [132]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i : s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'[,.:;?_!"()\']|--|\s', text)

        preprocessed = [item.strip() for item in preprocessed if item and item.strip()]

        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
        
        ids = [self.str_to_int[s] for s in preprocessed]

        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+(,.?!"()\'])', r'\1', text)
        return text

In [133]:
tokenizer = SimpleTokenizerV2(vocab)
text = "Hello, do you like tea. is this-- a test?"

tokenizer.encode(text)


# Now it has no error as previous problem that we faced in the tokenizing the 'Hello' 

[1159, 386, 1154, 659, 1002, 615, 1026, 146, 1159]

In [134]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|> do you like tea is this a <|unk|>'

### **2.4 Byte pair encoding**  - this is actually one of the most used 

In [135]:
import tiktoken

In [136]:
tiktoken.__version__

'0.12.0'

In [137]:
tokenizer = tiktoken.get_encoding("gpt2")

In [138]:
tokenizer.encode("Hello world!")

[15496, 995, 0]

In [139]:
tokenizer.decode(tokenizer.encode("Hello world!"))

'Hello world!'

In [162]:
tokenizer.n_vocab

50257

In [140]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
        "of someunknownPlace.")                           # <|endoftext|> USUALLY USED TO CONCATINATE TEXT

tokenizer.encode(text, allowed_special={"<|endoftext|>"})

[15496,
 11,
 466,
 345,
 588,
 8887,
 30,
 220,
 50256,
 554,
 262,
 4252,
 18250,
 8812,
 2114,
 1659,
 617,
 34680,
 27271,
 13]

### **2.5 Data Sampling With Sliding Window**
* Our task is to train in way that it predict the next wordings

In [141]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)  # here we are using TIKTOKEN to tokanise
len(enc_text)


5145

In [142]:
enc_sample =  enc_text[50:]
len(enc_sample)

5095

In [143]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f" x : {x}")
print(f" y :       {y}")

 x : [290, 4920, 2241, 287]
 y :       [4920, 2241, 287, 257]


In [144]:
# more example
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "------->", desired)

[290] -------> 4920
[290, 4920] -------> 2241
[290, 4920, 2241] -------> 287
[290, 4920, 2241, 287] -------> 257


In [145]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "------>", tokenizer.decode([desired]))

 and ------>  established
 and established ------>  himself
 and established himself ------>  in
 and established himself in ------>  a


In [146]:
import torch 
torch.__version__

'2.9.0+cpu'

In [147]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, strides):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        #Use a sliding window to chunk the book into overlapping sequences of max_length 
        for i in range(0, len(token_ids) - max_length, strides):
            input_chunk = token_ids[i:i + max_length + 1]
            target_chunk = token_ids[i+1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
            

In [148]:
def create_dataloader_v1(txt, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):

    # Initialize the tokenizer 
    tokenizer = tiktoken.get_encoding("gpt2")

    # Creaṭe dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [149]:
with open("the-verdict.txt", "r", encoding = "utf-8") as f:
    raw_text = f.read()

In [157]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length = 4, stride=4, shuffle = False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807]])]


In [158]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 1807,  3619,   402,   271, 10899]]), tensor([[ 3619,   402,   271, 10899]])]


In [160]:
dataloader = create_dataloader_v1(raw_text, batch_size=2, max_length=4, stride=4, shuffle=False)

for inputs, targets in dataloader:
    pass

print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  338,   645, 42393,   803,   674],
        [  674,  1611,   286,  1242,   526]])

Targets:
 tensor([[  645, 42393,   803,   674],
        [ 1611,   286,  1242,   526]])


### **2.7 Creating token embeddings**

In [161]:
input_ids = torch.tensor([2, 3, 5, 1])


In [163]:
# Let's create an embedding layer
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [165]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)
